The goal of this notebook is to tidy up the BN10 metabolomics data. This data is to find donors with high SCFA and secondary bile acids. This is for donor selection for Jas's liver cirrhosis FMT study.

In [1]:
import pandas as pd
import feather

First, let's read in all the data, convert each sheet into tidy data, and then concatenate them into one clean dataframe.

C-18 neg has the bile acids.

In [2]:
fname = '../../data/raw/bn10/16_1028_BN10_openbiome_MxP.xlsx'
c18neg = pd.read_excel(fname, sheet_name='C18-neg', skiprows=1)

# Tidy dataframe
ftidy = '../../data/clean/bn10.tidy_metabolomics.old_data.feather'

In [3]:
c18neg.shape

(13170, 186)

In [4]:
c18neg.head()

,Method,Compound,m/z,RT (min),HMDB ID (*representative ID),Metabolite,0001-0163,0005-0140,0014-0002,0025-0002,...,0544-0001,0544-0008,0549-0001,0549-0010,0571-0001,0571-0009,0572-0001,0572-0016,0574-0001,0574-0003
0,C18-neg,cmp.QI01,355.242367,7.757533,Internal Standard,PGE2-d4,289686.0,290595.0,302638.0,272843.0,...,296533.0,294045.0,286796.0,319931.0,298972.0,292705.0,300217.0,303752.0,299045.0,289756.0
1,C18-neg,cmp.QI02,313.238162,9.830450,HMDB04705,12_13-diHOME,191459.0,1256526.0,639131.0,112859.0,...,186940.0,228675.0,471212.0,597694.0,147788.0,94566.0,1323674.0,342138.0,277745.0,206769.0
2,C18-neg,cmp.QI03,313.238234,10.023333,HMDB04704,9_10-diHOME,196193.0,940243.0,788211.0,87221.0,...,150893.0,194845.0,354305.0,398334.0,114068.0,166261.0,796338.0,454767.0,414934.0,323602.0
3,C18-neg,cmp.QI04,295.227428,11.289300,HMDB04667,13-HODE,82445.0,1043574.0,95708.0,296132.0,...,77153.0,258187.0,182365.0,152416.0,28902.0,98580.0,355470.0,722006.0,108549.0,147500.0
4,C18-neg,cmp.QI05,335.222620,11.003983,NaN,5_6 diHETE,32595.0,7213.0,25398.0,68552.0,...,12800.0,16872.0,4310.0,8021.0,1838.0,21655.0,189.0,1360.0,6891.0,26992.0


Data is in a hybrid tidy and non-tidy data format. Let's transform it to tidy (I guess?)

In [5]:
c18neg.columns

Index([u'Method', u'Compound', u'm/z', u'RT (min)',
       u'HMDB ID (*representative ID)', u'Metabolite', u'0001-0163',
       u'0005-0140', u'0014-0002', u'0025-0002',
       ...
       u'0544-0001', u'0544-0008', u'0549-0001', u'0549-0010', u'0571-0001',
       u'0571-0009', u'0572-0001', u'0572-0016', u'0574-0001', u'0574-0003'],
      dtype='object', length=186)

In [6]:
id_vars = ['Method', 'Compound', 'm/z', 'RT (min)',
           'HMDB ID (*representative ID)', 'Metabolite']
tidyc18 = pd.melt(c18neg, id_vars=id_vars, var_name='sample', value_name='intensity')

In [7]:
tidyc18.shape

(2370600, 8)

In [8]:
tidyc18.head()

,Method,Compound,m/z,RT (min),HMDB ID (*representative ID),Metabolite,sample,intensity
0,C18-neg,cmp.QI01,355.242367,7.757533,Internal Standard,PGE2-d4,0001-0163,289686.0
1,C18-neg,cmp.QI02,313.238162,9.830450,HMDB04705,12_13-diHOME,0001-0163,191459.0
2,C18-neg,cmp.QI03,313.238234,10.023333,HMDB04704,9_10-diHOME,0001-0163,196193.0
3,C18-neg,cmp.QI04,295.227428,11.289300,HMDB04667,13-HODE,0001-0163,82445.0
4,C18-neg,cmp.QI05,335.222620,11.003983,NaN,5_6 diHETE,0001-0163,32595.0


In [9]:
tidyc18['intensity'].isna().sum()

196215

In [10]:
tidyc18['intensity'] = tidyc18['intensity'].fillna(0.0)

HILIC-neg sheet has the SCFAs

In [11]:
hilicneg = pd.read_excel(fname, sheet_name='HILIC-neg', skiprows=1)

In [12]:
hilicneg.columns

Index([u'Method', u'Compound ID', u'm/z', u'RT (min)',
       u'HMDB ID (*representative)', u'Metabolite', u'0001-0163', u'0005-0140',
       u'0014-0002', u'0025-0002',
       ...
       u'0544-0001', u'0544-0008', u'0549-0001', u'0549-0010', u'0571-0001',
       u'0571-0009', u'0572-0001', u'0572-0016', u'0574-0001', u'0574-0003'],
      dtype='object', length=186)

In [13]:
print(hilicneg.shape)
id_vars = ['Method', 'Compound ID', 'm/z', 'RT (min)',
           'HMDB ID (*representative)', 'Metabolite']
tidyhilic = pd.melt(hilicneg, id_vars=id_vars, var_name='sample', value_name='intensity')
tidyhilic = tidyhilic.rename(columns={'Compound ID': 'Compound',
                                      'HMDB ID (*representative)': 'HMDB ID (*representative ID)'})
tidyhilic['intensity'] = tidyhilic['intensity'].fillna(0.0)
print(tidyhilic.shape)

(10203, 186)
(1836540, 8)


In [14]:
tidyhilic.head()

,Method,Compound,m/z,RT (min),HMDB ID (*representative ID),Metabolite,sample,intensity
0,HILIC-neg,cmp.TF86,468.326868,3.90,Internal Standard,glycocholate-d4,0001-0163,5929344.0
1,HILIC-neg,cmp.TF166,129.060758,1.35,Internal Standard,thymine-d4,0001-0163,64564945.0
2,HILIC-neg,cmp.TF1,160.061531,5.25,HMDB00510,2-aminoadipate,0001-0163,2658543.0
3,HILIC-neg,cmp.TF2,168.030231,3.80,HMDB00439,2-furoylglycine,0001-0163,448600.0
4,HILIC-neg,cmp.TF4,147.029897,7.00,HMDB00694,2-hydroxyglutarate,0001-0163,19722832.0


In [15]:
hilicpos = pd.read_excel(fname, sheet_name='HILIC-pos', skiprows=1)
print(hilicpos.shape)
print(hilicpos.columns)

(17792, 186)
Index([u'Method', u'Compound', u'm/z', u'RT (min)',
       u'HMDB ID (*representative ID)', u'Metabolite', u'0001-0163',
       u'0005-0140', u'0014-0002', u'0025-0002',
       ...
       u'0544-0001', u'0544-0008', u'0549-0001', u'0549-0010', u'0571-0001',
       u'0571-0009', u'0572-0001', u'0572-0016', u'0574-0001', u'0574-0003'],
      dtype='object', length=186)


In [16]:
id_vars = ['Method', 'Compound', 'm/z', 'RT (min)',
           'HMDB ID (*representative ID)', 'Metabolite']
tidyhilicpos = pd.melt(hilicpos, id_vars=id_vars, var_name='sample', value_name='intensity')
tidyhilicpos['intensity'] = tidyhilicpos['intensity'].fillna(0.0)
print(tidyhilicpos.shape)

(3202560, 8)


In [17]:
tidyhilicpos.head()

,Method,Compound,m/z,RT (min),HMDB ID (*representative ID),Metabolite,sample,intensity
0,HILIC-pos,cmp.QI144,126.136488,7.316333,Internal Standard,valine-d8,0001-0163,60311.0
1,HILIC-pos,cmp.QI118,174.136519,6.609700,Internal Standard,phenylalanine-d8,0001-0163,166033.0
2,HILIC-pos,cmp.QI82,76.039339,7.822333,HMDB00123,glycine,0001-0163,641.0
3,HILIC-pos,cmp.QI23,90.054971,7.700183,HMDB00161,alanine,0001-0163,6781.0
4,HILIC-pos,cmp.QI130,106.049833,7.578050,HMDB00187,serine,0001-0163,3315.0


Concatenate them all.

In [18]:
tidyall = pd.concat((tidyc18, tidyhilic, tidyhilicpos))
print(tidyall.shape)
tidyall.head()

(7409700, 8)


,Method,Compound,m/z,RT (min),HMDB ID (*representative ID),Metabolite,sample,intensity
0,C18-neg,cmp.QI01,355.242367,7.757533,Internal Standard,PGE2-d4,0001-0163,289686.0
1,C18-neg,cmp.QI02,313.238162,9.830450,HMDB04705,12_13-diHOME,0001-0163,191459.0
2,C18-neg,cmp.QI03,313.238234,10.023333,HMDB04704,9_10-diHOME,0001-0163,196193.0
3,C18-neg,cmp.QI04,295.227428,11.289300,HMDB04667,13-HODE,0001-0163,82445.0
4,C18-neg,cmp.QI05,335.222620,11.003983,NaN,5_6 diHETE,0001-0163,32595.0


In [19]:
# Clean up column names
tidyall = tidyall.rename(
    columns={'Method': 'method', 'Compound': 'compound',
             'm/z': 'mz', 'RT (min)': 'rt',
             'HMDB ID (*representative ID)': 'hmdb_id',
             'Metabolite': 'metabolite'})
tidyall.head()

,method,compound,mz,rt,hmdb_id,metabolite,sample,intensity
0,C18-neg,cmp.QI01,355.242367,7.757533,Internal Standard,PGE2-d4,0001-0163,289686.0
1,C18-neg,cmp.QI02,313.238162,9.830450,HMDB04705,12_13-diHOME,0001-0163,191459.0
2,C18-neg,cmp.QI03,313.238234,10.023333,HMDB04704,9_10-diHOME,0001-0163,196193.0
3,C18-neg,cmp.QI04,295.227428,11.289300,HMDB04667,13-HODE,0001-0163,82445.0
4,C18-neg,cmp.QI05,335.222620,11.003983,NaN,5_6 diHETE,0001-0163,32595.0


Let's split the "sample" column into "donor ID" and "sample_number"

In [20]:
tidyall[['donor', 'sample_number']] = tidyall['sample'].str.split('-', expand=True)

In [21]:
tidyall.head()

,method,compound,mz,rt,hmdb_id,metabolite,sample,intensity,donor,sample_number
0,C18-neg,cmp.QI01,355.242367,7.757533,Internal Standard,PGE2-d4,0001-0163,289686.0,0001,0163
1,C18-neg,cmp.QI02,313.238162,9.830450,HMDB04705,12_13-diHOME,0001-0163,191459.0,0001,0163
2,C18-neg,cmp.QI03,313.238234,10.023333,HMDB04704,9_10-diHOME,0001-0163,196193.0,0001,0163
3,C18-neg,cmp.QI04,295.227428,11.289300,HMDB04667,13-HODE,0001-0163,82445.0,0001,0163
4,C18-neg,cmp.QI05,335.222620,11.003983,NaN,5_6 diHETE,0001-0163,32595.0,0001,0163


This is a very large dataframe. Let's use feather format to read/write the tidy data.

In [22]:
feather.write_dataframe(tidyall, ftidy)